Stand: 24.02.25

## Darstellung der Messtationsorte auf einer Karte
- 6 verschiedene Orte für den DWD der Station 02712 (blaue Marker)
    - ab dem 19.10.2020 wurde die Station von der Stadt ins Grüne verlegt
    - ID Klimamessstation DWD: 02712 (https://opendata.dwd.de/climate_environment/CDC/observations_germany/) 
- 1 Luftgütemessstation in der Stadt (roter Marker)
- 3 eventuell interessante Messstationen in der Schweiz (grüne Marker)
    - Zugriff auf die Daten eventuell erst ab 01.04.25 möglich, weil noch an der Webseite gearbetet wird

In [1]:
import folium
import pandas as pd
from datetime import datetime
import webbrowser
from pathlib import Path



# Daten laden
data_dwd = {
    "Stations_id": [2712, 2712, 2712, 2712, 2712, 2712],
    "Stationshoehe": [400.00, 398.00, 443.00, 442.50, 442.50, 428.06],
    "Geogr.Breite": [47.6594, 47.6672, 47.6774, 47.6774, 47.6774, 47.6952],
    "Geogr.Laenge": [9.1790, 9.1798, 9.1900, 9.1900, 9.1900, 9.1307],
    "von_datum": ["19510702", "19570801", "19720918", "20070307", "20180927", "20201019"],
    "bis_datum": ["19570731", "19720917", "20070306", "20180926", "20201018", ""],
    "Stationsname": ["Konstanz"] * 6
}

# Kreuzlingen - Daten seit 2010
# HAI - Daten seit 1959
# GUT - Daten seit 1976 - https://www.meteoschweiz.admin.ch/service-und-publikationen/applikationen/messwerte-und-messnetze.html#param=messnetz-automatisch&table=false&station=GUT
data_schweiz = {
    "Stationshoehe": [410, 720, 442],
    "Geogr.Breite": [47.647083, 47.651242, 47.601733],
    "Geogr.Laenge": [9.189364, 9.023911, 9.279428],
    "von_datum": ["01.09.2010", "01.01.1959", "01.06.1976"],
    "bis_datum": ["heute", "heute", "heute"],
    "Stationsname": ["Kreuzlingen", "Salen-Reutenen (HAI)", "Güttingen (GUT)"] 
}


df = pd.DataFrame(data_dwd)
df_schweiz = pd.DataFrame(data_schweiz)

# Datum ins deutsche Format umwandeln
def format_date(date_str):
    if date_str.strip() == "":
        return "heute"
    return datetime.strptime(date_str, "%Y%m%d").strftime("%d.%m.%Y")


# Karte erstellen, zentriert auf den Mittelwert der Koordinaten
m = folium.Map(location=[df["Geogr.Breite"].mean(), df["Geogr.Laenge"].mean()], zoom_start=12)

# DataFrame nach dem frühesten 'von_datum' sortieren
df_sorted = df.sort_values(by='von_datum')

# Gruppierung der Daten nach den Koordinaten
grouped = df_sorted.groupby(['Geogr.Breite', 'Geogr.Laenge'])

# Marker mit Tooltip, Popup und blauen Zahlen hinzufügen
counter = 1
for (lat, lon), group in grouped:
    # Popup-Text für alle Datensätze an derselben Position
    popup_text = "<br>".join([
        f"DWD<br>Von: {format_date(row['von_datum'])} - Bis: {format_date(row['bis_datum'])} <br>Höhe: {row["Stationshoehe"]}" 
        for _, row in group.iterrows() 
    ])
    
    popup = folium.Popup(popup_text, max_width=300, close_button=True)
    marker = folium.Marker(
        location=[lat, lon],
        tooltip=popup_text
    )
    marker.add_to(m)
    marker.add_child(popup)
    popup.show = True  # Popup von Anfang an sichtbar machen

    # Blaue Zahlen nebeneinander anzeige
    number_html = ''.join([
        f'<div style="display:inline-block; font-size: 16pt; color: blue; font-weight: bold; margin-right: 5px;">{i}</div>' 
        for i in range(counter, counter + len(group))
    ])

    folium.map.Marker(
        [lat + 0.003, lon + 0.004],
        icon=folium.DivIcon(html=number_html)
    ).add_to(m)

    counter += len(group)


# ! Luftgüte-Station (rote Markierung) hinzufügen

# WGS84 Koordinaten: 9°10'09.44" E, 47°39'51.70" N
luftguete_lat = 47 + 39/60 + 51.70/3600  # Umrechnung in Dezimalgrad
luftguete_lon = 9 + 10/60 + 9.44/3600    # Umrechnung in Dezimalgrad

popup_luftguete = folium.Popup("Luftgüte", max_width=300, close_button=True)
marker_luftguete = folium.Marker(
    location=[luftguete_lat, luftguete_lon],
    tooltip="Luftgüte",
    icon=folium.Icon(color='red')
)
marker_luftguete.add_to(m)
marker_luftguete.add_child(popup_luftguete)
popup_luftguete.show = True  # Popup von Anfang an sichtbar machen

for index, row in df_schweiz.iterrows():
    
    
    popup_text = f"{row["Stationsname"]}<br>Von: {row['von_datum']} - Bis: {row['bis_datum']} <br>Höhe: {row["Stationshoehe"]}"

    
    popup = folium.Popup(popup_text, max_width=300, close_button=True)

    marker = folium.Marker(
        location=[row["Geogr.Breite"], row["Geogr.Laenge"]],
        tooltip=f"Schweiz: {row["Stationsname"]}",
        icon=folium.Icon(color='green')
    )
    marker.add_to(m)
    marker.add_child(popup)
    popup.show = True  # Popup von Anfang an sichtbar machen


# Ordner 'data' erstellen, falls nicht vorhanden
data_dir = Path.cwd() / "../data"
data_dir.mkdir(parents=True, exist_ok=True)

# Karte speichern und im Browser öffnen
map_file = data_dir / "konstanz_map.html"
m.save(map_file)

# Karte im Standardbrowser öffnen
webbrowser.open(map_file.as_uri())


True